<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C3-Owning_The_Network_With_Scapy/ARP_Cache_Poisoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__ARP Cache Poisoning with Scapy__*

ARP poisoning is one of the oldest yet most effective tricks in a hacker's toolkit. Quite simply, we will convince a target machine that we have become its gateway, and we will also convince the gateway that in order to reach the target machine, all traffic has to go through us.